In [ ]:
import os
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
# matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np

from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

from load_paus_cat import load_paus_cat
from paus_utils import w_central, z_NB

import pandas as pd

In [ ]:
vi_cat = fits.open('/home/alberto/almacen/PAUS_data/catalogs/PAUS_LAE_selection_visual_insp_AT.fits')[1].data
vi_cat_hiz = fits.open('/home/alberto/almacen/PAUS_data/catalogs/LAE_selection_VI_hiZ.fits')[1].data

In [ ]:
paus_tcurves_dir = '/home/alberto/almacen/PAUS_data/OUT_FILTERS'
tcurves_file_list = os.listdir(paus_tcurves_dir)
tcurves_file_list.sort()

paus_fil_names = []

for name in tcurves_file_list:
    if name[4] == 'D':
        this_name = f'NB{name[6:9]}'
    else:
        this_name = name[-5]

    paus_fil_names.append(this_name)

paus_fil_names = paus_fil_names[6:]

In [ ]:
# Initialize output data dict
to_stack_data = {}
for nbi in range(40):
    to_stack_data[paus_fil_names[nbi]] = []
    to_stack_data[f'{paus_fil_names[nbi]}_error'] = []
to_stack_data['z'] = []
to_stack_lya_NB = []
to_stack_L_lya = []


for field_name in ['W1', 'W2', 'W3']:
    path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}_extinction_corrected.pq']
    cat = load_paus_cat(path_to_cat)

    # Get IDs from LAEs visually selected
    # mask = ~vi_cat['is_junk_VI'] & (vi_cat['field'] == field_name)
    mask = vi_cat['is_LAE_VI'] & (vi_cat['field'] == field_name)
    # mask = vi_cat['is_hiZ_LAE'] & (vi_cat['field'] == field_name)


    LAE_vi_IDs = np.array(vi_cat['ref_id'][mask])

    lya_NB = np.array(vi_cat['lya_NB'][mask])

    redshifts  = np.array(vi_cat['z_NB'][mask])
    # redshifts = z_NB(lya_NB)
    L_lya  = np.array(vi_cat['L_lya_corr'][mask])
    print(len(LAE_vi_IDs))

    where_LAEs_in_cat = np.empty_like(LAE_vi_IDs).astype(int)
    for i, thisid in enumerate(LAE_vi_IDs):
        where_LAEs_in_cat[i] = np.where(thisid == cat['ref_id'])[0][0]

    # Compute normalization for each source, in the 160-180 nm band (rest-frame)
    norm = np.empty_like(L_lya)
    norm_wl_min, norm_wl_max = 1300, 1365

    for src in range(len(L_lya)):
        mask_norm_band = (w_central > norm_wl_min * (1 + redshifts[src])) & (w_central < norm_wl_max * (1 + redshifts[src]))
        mask_norm_band[-6:] = False
        dl = cosmo.luminosity_distance(redshifts[src]).to(u.cm).value
        norm[src] = np.average(cat['flx'][mask_norm_band, where_LAEs_in_cat[src]] * 1450 * (1 + redshifts[src]) * dl**2,
                               weights=(cat['err'][mask_norm_band, where_LAEs_in_cat[src]] * 1450 * (1 + redshifts[src]) * dl**2)**-2)

    # Data to save to a .csv to be read by stonp
    # 40 Narrow-bands
    for nbi in range(40):
        to_stack_data[paus_fil_names[nbi]].append(cat['flx'][nbi, where_LAEs_in_cat] / norm)
        to_stack_data[f'{paus_fil_names[nbi]}_error'].append(cat['err'][nbi, where_LAEs_in_cat] / norm)
    to_stack_data['z'].append(redshifts)
    to_stack_lya_NB.append(lya_NB)
    to_stack_L_lya.append(L_lya)

for field_name in ['W1', 'W2', 'W3']:
    path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}_extinction_corrected.pq']
    cat = load_paus_cat(path_to_cat)

    # Get IDs from LAEs visually selected
    mask = vi_cat_hiz['is_hiZ_LAE'] & (vi_cat_hiz['field'] == field_name)


    LAE_vi_IDs = np.array(vi_cat_hiz['ref_id'][mask])

    lya_NB = np.array(vi_cat_hiz['lya_NB'][mask])
    lya_NB[vi_cat_hiz['lya_NB_VI'][mask] > 0] = vi_cat_hiz['lya_NB_VI'][mask][vi_cat_hiz['lya_NB_VI'][mask] > 0]


    redshifts = z_NB(lya_NB)
    L_lya  = np.array(vi_cat_hiz['L_lya_corr'][mask])
    print(len(LAE_vi_IDs))

    where_LAEs_in_cat = np.empty_like(LAE_vi_IDs).astype(int)
    for i, thisid in enumerate(LAE_vi_IDs):
        where_LAEs_in_cat[i] = np.where(thisid == cat['ref_id'])[0][0]

    # Compute normalization for each source, in the 160-180 nm band (rest-frame)
    norm = np.empty_like(L_lya)
    norm_wl_min, norm_wl_max = 1300, 1365

    for src in range(len(L_lya)):
        mask_norm_band = (w_central > norm_wl_min * (1 + redshifts[src])) & (w_central < norm_wl_max * (1 + redshifts[src]))
        mask_norm_band[-6:] = False
        dl = cosmo.luminosity_distance(redshifts[src]).to(u.cm).value
        norm[src] = np.average(cat['flx'][mask_norm_band, where_LAEs_in_cat[src]] * 1450 * (1 + redshifts[src]) * dl**2,
                               weights=(cat['err'][mask_norm_band, where_LAEs_in_cat[src]] * 1450 * (1 + redshifts[src]) * dl**2)**-2)

    # Data to save to a .csv to be read by stonp
    # 40 Narrow-bands
    for nbi in range(40):
        to_stack_data[paus_fil_names[nbi]].append(cat['flx'][nbi, where_LAEs_in_cat] / norm)
        to_stack_data[f'{paus_fil_names[nbi]}_error'].append(cat['err'][nbi, where_LAEs_in_cat] / norm)
    to_stack_data['z'].append(redshifts)
    to_stack_lya_NB.append(lya_NB)
    to_stack_L_lya.append(L_lya)


for nbi in range(40):
    to_stack_data[paus_fil_names[nbi]] = np.concatenate(to_stack_data[paus_fil_names[nbi]])
    to_stack_data[f'{paus_fil_names[nbi]}_error'] = np.concatenate(to_stack_data[f'{paus_fil_names[nbi]}_error'])
to_stack_data['z'] = np.concatenate(to_stack_data['z'])
to_stack_lya_NB = np.concatenate(to_stack_lya_NB)
to_stack_L_lya = np.concatenate(to_stack_L_lya)

In [ ]:
import stonp

# Save it to csv
nb_list = [[0, 4], [3, 7], [6, 10], [9, 13], [12, 16], [14, 19], [19, 25], [23, 50]]
# nb_list = [[3, 18]]


wl_list = []
stacked_seds_50 = []
stacked_seds_16 = []
stacked_seds_84 = []
stacked_seds_err = []

common_wl_grid = np.linspace(70, 250, 500)

N_boots = 100
for [nb1, nb2] in nb_list:
    this_mask = (to_stack_lya_NB >= nb1) & (to_stack_lya_NB <= nb2) & (to_stack_L_lya > 43.5)
    print(sum(this_mask))

    this_df = pd.DataFrame(to_stack_data)[this_mask]
    this_stack_50_list = []
    for boot_i in range(N_boots):
    # Bootstrap sources, save them as csv and stack
        boot_ids = np.random.choice(np.arange(len(this_df)), len(this_df), replace=True)

        df_to_save = this_df.iloc[boot_ids]
        df_to_save.reset_index()

        df_to_save.to_csv(f'fluxes_to_stack_NB{nb1}_{nb2}.csv')


        st = stonp.Stacker()

        st.load_catalog(f'fluxes_to_stack_NB{nb1}_{nb2}.csv', z_label='z')
        # st.flux_units = u.dimensionless_unscaled
        st.to_rest_frame(flux_conversion='luminosity', wl_obs_min=440, wl_obs_max=850)
        st.stack(error_type='flux_error')
        stack_50 = st.stacked_seds
        stack_err = st.stack_sed_err

        this_stack_50_list.append(np.interp(common_wl_grid, st.wl_grid, stack_50[0]))


    wl_list.append(common_wl_grid)
    stacked_seds_50.append(np.nanmean(this_stack_50_list, axis=0))
    stacked_seds_16.append(np.nanpercentile(this_stack_50_list, axis=0, q=16))
    stacked_seds_84.append(np.nanpercentile(this_stack_50_list, axis=0, q=84))
    stacked_seds_err.append(np.nanstd(this_stack_50_list, axis=0))

In [ ]:
# All
this_mask = (to_stack_lya_NB >= 0) & (to_stack_lya_NB <= 30) & (to_stack_L_lya > 44)
print(sum(this_mask))

this_df = pd.DataFrame(to_stack_data)[this_mask]
this_stack_50_list = []
for boot_i in range(N_boots):
# Bootstrap sources, save them as csv and stack
    boot_ids = np.random.choice(np.arange(len(this_df)), len(this_df), replace=True)

    df_to_save = this_df.iloc[boot_ids]
    df_to_save.reset_index()

    df_to_save.to_csv(f'fluxes_to_stack_NB{nb1}_{nb2}.csv')


    st = stonp.Stacker()

    st.load_catalog(f'fluxes_to_stack_NB{nb1}_{nb2}.csv', z_label='z')
    st.to_rest_frame(flux_conversion='luminosity', wl_obs_min=440, wl_obs_max=850)
    st.stack(error_type='flux_error')
    stack_50 = st.stacked_seds
    stack_err = st.stack_sed_err

    this_stack_50_list.append(np.interp(common_wl_grid, st.wl_grid, stack_50[0]))

sed_50_all = np.nanmean(this_stack_50_list, axis=0)
sed_err_all = (np.nanpercentile(this_stack_50_list, axis=0, q=84) - np.nanpercentile(this_stack_50_list, axis=0, q=16)) * 0.5
wl_all = common_wl_grid

In [ ]:
from uncertainties import ufloat
from scipy.optimize import curve_fit

def QSO_cont(x, a, b):
    return b * x**a

z_list = []
z_50 = []
z_16 = []
z_84 = []
meas_flux_list = []
meas_flux_16_list = []
meas_flux_84_list = []
pred_flux_list = []

iii = 0
for wl, sed_50, sed_16, sed_84 in zip(wl_list, stacked_seds_50, stacked_seds_16, stacked_seds_84):
    this_mask = (to_stack_lya_NB >= nb_list[iii][0]) & (to_stack_lya_NB <= nb_list[iii][1])
    this_z_mean = np.mean(to_stack_data['z'][this_mask])
    wl_mask_fit = ((
                    ((wl > 135) & (wl < 136.4))
                    | ((wl > 145) & (wl < 148))
                    # | ((wl > 177) & (wl < 182))
                    # | ((wl > 200)) & (wl < 225)
                    )
                    & np.isfinite(sed_50))

    try:
        popt, pcov = curve_fit(QSO_cont, wl[wl_mask_fit], sed_50[wl_mask_fit],)
                            #    sigma=sed_err[wl_mask_fit])
    except:
        print('Fit err')
        continue

    fig, ax = plt.subplots(figsize=(12, 5))

    ax.errorbar(wl[wl_mask_fit], sed_50[wl_mask_fit], fmt='o')
    ax.plot(wl, sed_50)

    xx = np.linspace(100, 215, 1000)
    ax.plot(xx, QSO_cont(xx, popt[0], popt[1]))


    mask_lyforest = (wl > 113) & (wl < 115)
    ax.plot(wl[mask_lyforest], sed_50[mask_lyforest], marker='o')

    ax.axhline(0, c='k')
    # ax.set_yscale('log')
    # ax.set_ylim(1e-3)

    plt.show()

    pred_flux = QSO_cont(113.5, ufloat(popt[0], pcov[0, 0]**0.5), ufloat(popt[1], pcov[1, 1]**0.5))
    meas_flux = np.nanmean(sed_50[mask_lyforest])
    meas_flux_16 = np.nanmean(sed_16[mask_lyforest])
    meas_flux_84 = np.nanmean(sed_84[mask_lyforest])
    # meas_flux_16 = np.nanpercentile(sed_50[mask_lyforest], q=16)
    # meas_flux_84 = np.nanpercentile(sed_50[mask_lyforest], q=84)

    LyF_trans = ufloat(meas_flux, (meas_flux_84 - meas_flux_16) * 0.5) / pred_flux
    print(LyF_trans)

    z_list.append(this_z_mean)
    z_50.append(np.median(to_stack_data['z'][this_mask]))
    z_16.append(np.percentile(to_stack_data['z'][this_mask], q=16))
    z_84.append(np.percentile(to_stack_data['z'][this_mask], q=84))

    meas_flux_list.append(meas_flux)
    meas_flux_16_list.append(meas_flux_16)
    meas_flux_84_list.append(meas_flux_84)
    pred_flux_list.append(pred_flux)

    iii += 1

z_50 = np.array(z_50)
z_16 = np.array(z_16)
z_84 = np.array(z_84)

In [ ]:
import lmfit

# sed_50 = stacked_seds_50[0]
# sed_err = stacked_seds_err[0]
# wl = wl_list[0]
sed_50 = np.array(sed_50_all)
sed_err = np.array(sed_err_all)
wl = wl_all

this_z_mean = np.mean(to_stack_data['z'])
wl_mask_fit = ((
                ((wl > 134) & (wl < 135))
                | ((wl > 146) & (wl < 147))
                # | ((wl > 169) & (wl < 170))
                # | ((wl > 200)) & (wl < 225)
                )
                & np.isfinite(sed_50) & np.isfinite(sed_err))


def QSO_cont_fit(x, a, b):
    return np.log10(b * x**a)

model = lmfit.Model(QSO_cont_fit)
model.set_param_hint('a', value=-1.3, min=-4, max=0)
model.set_param_hint('b', value=8, min=0, max=500)

sed_to_fit = list(sed_50[wl_mask_fit]) * 100
wl_to_fit = list(wl[wl_mask_fit]) * 100
err_to_fit = list(sed_err[wl_mask_fit]) * 100
sed_to_fit += np.random.normal(size=len(sed_to_fit)) * np.array(err_to_fit)

result = model.fit(np.log10(sed_to_fit), x=wl_to_fit,
                   weights=(np.array(err_to_fit))**-2,
                   method='leastsq', nan_policy='propagate')
print(result.fit_report())


fig, ax = plt.subplots(figsize=(6, 5))

norm = np.nanmax(sed_50)

# ax.errorbar(wl[wl_mask_fit], sed_50[wl_mask_fit] / norm,
#             fmt='o', yerr=sed_err[wl_mask_fit])
ax.plot(wl, sed_50 / norm,
        lw=2, c='teal')

xx = np.linspace(92, 300, 1000)
ax.plot(xx, QSO_cont(xx, result.params['a'], result.params['b']) / norm,
        lw=2, zorder=-2314098, alpha=1, c='r', ls='--')


# ax.fill_between([113, 115], [-1, -1], [100, 100],
#                 color='orange', alpha=0.6, zorder=-99, lw=0)

ax.fill_between([134, 135], [-1, -1], [100, 100],
                color='dimgray', alpha=0.6, zorder=-99, lw=0)
ax.fill_between([146, 147], [-1, -1], [100, 100],
                color='dimgray', alpha=0.6, zorder=-99, lw=0)

ax.text(92, 0.13, r'LyC', fontsize=12)
ax.text(103.1, 0.45, r'Ly$\beta$' + '\n+OVI', fontsize=12)
ax.text(122.1, 1, r'Ly$\alpha$' + '\n+NV', fontsize=12)
ax.text(135, 0.35, 'SiIV\n+OIV', fontsize=12)
ax.text(156, 0.55, 'CIV', fontsize=12)
ax.text(192, 0.3, 'CIII', fontsize=12)


ax.axvline(91.2, ls=':', c='dimgray', zorder=-99)

ax.set_xscale('log')
ax.set_yscale('log')
# ax.set_xticks(np.arange(50, 230, 10))

ax.set_ylim(0.1, 1.25)
ax.set_xlim(85, 220)

ax.set_ylabel(r'$L_\lambda$ [A. U.]')
ax.set_xlabel('Rest-frame wavelength [nm]')

ax.tick_params(labelsize=17, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.savefig('../figures/stacked_QSO_all.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()
pred_flux_all = ufloat(result.eval(x=114), result.eval_uncertainty(x=[114])[0])
print(pred_flux_all)
meas_flux_all = np.average(sed_50[mask_lyforest])

In [ ]:
ref_lum = np.load('ref_lum.npy')
ref_wav = np.load('ref_wav.npy')

fig, ax = plt.subplots(figsize=(12, 4.5))

cmap = plt.get_cmap('rainbow')
norm = np.nanmax(stacked_seds_50[3])

iii = 0
for wl, sed_50 in zip(wl_list, stacked_seds_50):
    this_mask = (to_stack_lya_NB >= nb_list[iii][0]) & (to_stack_lya_NB <= nb_list[iii][1])
    this_z_mean = np.mean(to_stack_data['z'][this_mask])

    ax.plot(wl, sed_50 / norm,
            c=cmap((this_z_mean - 3) / (5 - 3)),
            lw=2, zorder=90-iii)
    ax.plot([], [],
            label=r'$\bar{z}=$ ' + f'{this_z_mean:0.2f}',
            c=cmap((this_z_mean - 3) / (5 - 3)),
            lw=4)
    iii += 1

xx = np.linspace(91, 215, 1000)
print(f'UV slope = {result.params['b'].value:0.2f} +/- {result.params['a'].stderr**0.5:0.2f}')
ax.plot(xx, QSO_cont(xx, result.params['a'].value, result.params['b'].value) / norm,
        lw=2, zorder=-2314098, alpha=0.5, c='r', ls='--')


ax.text(92, -0.2, r'LyC')
ax.text(103.1, -0.2, r'Ly$\beta$')
ax.text(122.1, -0.2, r'Ly$\alpha$')
ax.text(141, -0.2, 'SiIV\n+OIV')
ax.text(156, -0.2, 'CIV')
ax.text(192, -0.2, 'CIII')


ax.axvline(91.2, ls=':', c='dimgray', zorder=-99)
ax.axvline(154.9, ls=':', c='dimgray', zorder=-99)
ax.axvline(121.56, ls=':', c='dimgray', zorder=-99)
ax.axvline(102.572, ls=':', c='dimgray', zorder=-99)
ax.axvline(139.98, ls=':', c='dimgray', zorder=-99)
ax.axvline(190.8734, ls=':', c='dimgray', zorder=-99)

ax.axhline(0, c='k')

ax.set_xlabel(r'Rest-frame wavelength [nm]')
ax.set_ylabel(r'$L_\lambda$ [A. U.]')
ax.legend(ncol=2, fontsize=13, framealpha=1)
ax.set_xlim(75, 210)
ax.set_ylim(-0.25, 1.25)

ax.tick_params(labelsize=17, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')


fig.savefig('../figures/stacked_QSO_zbins.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

In [ ]:
def IGM_TRANSMISSION(redshift_Arr, A, B):

    Transmission_Arr = pow(np.e, A * (1 + redshift_Arr)**B)

    return Transmission_Arr

def FG_T(redshift_Arr):

    A = -0.001845
    B = 3.924

    Transmission = IGM_TRANSMISSION(redshift_Arr, A, B)

    return Transmission


fig, ax = plt.subplots(figsize=(6, 4))

T_Arr = []
for f, f16, f84 in zip(meas_flux_list, meas_flux_16_list, meas_flux_84_list):
    T_Arr.append(ufloat(f, (f84 - f16) * 0.5) / 10**pred_flux_all)


z_LyF = 113 * (z_50 + 1) / 121.567 - 1
z_LyF_16 = 113 * (z_16 + 1) / 121.567 - 1
z_LyF_84 = 113 * (z_84 + 1) / 121.567 - 1

ax.errorbar(z_LyF[1:], [t.n for t in T_Arr][1:],
            xerr=[(z_LyF - z_LyF_16)[1:], (z_LyF_84 - z_LyF)[1:]],
            yerr=[t.s for t in T_Arr][1:],
            fmt='s', capsize=2, mfc='lightseagreen',
            mec='k', ecolor='k', label='This work',
            zorder=9999, ms=6.5)


### Literature
# Faucher-Giguère+08
xx = np.linspace(0, 6, 1000)
ax.plot(xx, FG_T(xx), lw=2, ls='--', zorder=-1, color='darkblue',
        label='Faucher-Giguère+08 (best-fit)')

FG08 = pd.read_csv('/home/alberto/almacen/PAUS_data/T_Lya_forest/FG08.csv', header=None).to_numpy()
FG08_z = FG08[::2][:, 0]
FG08_T = np.e**-FG08[::2][:, 1]
FG08_T_err = np.e**-FG08[::2][:, 1] * (FG08[1::2][:, 1] - FG08[::2][:, 1])
ax.errorbar(FG08_z, FG08_T,
            yerr=FG08_T_err,
            fmt='o', mec='dimgray', mfc='orange',
            capsize=2, ecolor='orange',
            label='Faucher-Giguère+08')

# Becker+13
ax.plot(xx, np.e**-(0.751 * ((1 + xx) / (1 + 3.5))**2.9 - 0.132),
        lw=2, ls='--', zorder=-1, color='orangered',
        label='Becker+13 (best-fit)')

B13 = pd.read_csv('/home/alberto/almacen/PAUS_data/T_Lya_forest/B13.csv', header=None).to_numpy()
B13_z = B13[::2][:, 0]
B13_T = np.e**((-B13[1::2][:, 1] -B13[::2][:, 1]) *  0.5)
B13_T_err = -np.e**(-B13[::2][:, 1]) + np.e**-B13[1::2][:, 1]

ax.errorbar(B13_z, np.abs(B13_T),
            yerr=np.abs(B13_T_err),
            fmt='^', mec='dimgray', mfc='orchid',
            capsize=2, ecolor='orchid',
            label='Becker+13', ms=8)



ax.set_ylim(0, 1)
# ax.set_yscale('log')
ax.set_xlim(1, 6)

ax.set_xlabel('Redshift')
ax.set_ylabel('IGM transmission')

ax.tick_params(labelsize=17, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.legend(fontsize=11, frameon=False)

fig.savefig('../figures/IGM_Lya_forest_T.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()